In [2]:
import sounddevice as sd
import numpy as np
import tensorflow as tf
import time
import os
import scipy as sp

In [3]:
names = {
    0 : 'Джарвис',
    1 : 'Дио',
    2 : 'Итан',
    3 : 'Лада',
    4 : 'Мира',
    5 : 'Привет'
}

sd.default.samplerate = 44100
sd.default.channels = 1

rec_duration = 0.1
start_duration = 0.5

model = tf.keras.models.load_model('AudRec_L2_v1-2(69-60).h5')

In [69]:
def func(_x, _sr=44100):
    _x = np.reshape(_x, len(_x))
    coeff = sp.signal.firwin(999, [260, 700], fs=_sr, pass_zero=False)
    x_filtered = sp.signal.lfilter(coeff, 1.0, _x)
    x_normalized = x_filtered/x_filtered.max()
    x_squared = np.square(x_normalized)
    splited = np.array_split(x_squared, 20)
    e_parts = np.empty((0))
    for part in splited:
        e_parts = np.append(e_parts, sp.integrate.simps(part))
    return tf.keras.utils.normalize(e_parts)

In [70]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 8)                 168       
_________________________________________________________________
dense_15 (Dense)             (None, 6)                 54        
Total params: 222
Trainable params: 222
Non-trainable params: 0
_________________________________________________________________


In [64]:
data = sd.rec(int((start_duration + 0.1) * sd.default.samplerate * 2), blocking=True)

In [71]:
def callback(indata, outdata, frames, time, status):
    global data, model
    print(names[model.predict_classes(func(data))[0]])
    data = data[frames:]
    data = np.append(data, indata, axis=0)

block = int(sd.default.samplerate * rec_duration)
with sd.Stream(blocksize=block, callback=callback):
    sd.sleep(int(1000 * 5))

Джарвис
Джарвис
Привет
Итан
Итан
Итан
Итан
Итан
Итан
Дио
Итан
Дио
Джарвис
Лада
Итан
Итан
Итан
Привет
Лада
Привет
Привет
Итан
Итан
Итан
Итан
Итан
Итан
Итан
Итан
Итан
Итан
Привет
Привет
Привет
Привет
Привет
Привет
Привет
Привет
Привет
Привет
Привет
Привет
Привет
Привет
Итан
Итан
Итан
Итан


In [65]:
sd.play(data)

In [66]:
print(names[model.predict_classes(func(data))[0]])

Джарвис


In [4]:
def callback(indata, outdata, frames, time, status):
    outdata[:] = indata
    print(time.currentTime)

block = int(sd.default.samplerate * rec_duration)
stream = sd.Stream(blocksize=block, callback=callback)

In [9]:
stream.start()

In [12]:
stream.active

False

In [10]:
stream.stop()

In [15]:
stream.latency

(0.1, 0.2)